In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-267698
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-267698


In [2]:
import pandas as pd

dic_data = {'Workspace name': ws.name,
            'Azure region': ws.location,
            'Subscription id': ws.subscription_id,
            'Resource group': ws.resource_group,
            'Experiment Name': exp.name}

data_fr = pd.DataFrame.from_dict(data = dic_data, orient='index')
data_fr

,0
Workspace name,quick-starts-ws-267698
Azure region,southcentralus
Subscription id,cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group,aml-quickstarts-267698
Experiment Name,udacity-project


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Define Compute CPU cluster name
cluster_name = "udacity-project"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                           min_nodes=0, 
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpu-cluster


In [4]:
# Check details about compute_targets (i.e. cpu_cluster)

compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

udacity-project ComputeInstance Succeeded


In [7]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 0.2, 0.5, 0.7, 1.0),
    '--max_iter': choice(range(10,110,10))
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, 
                      slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    compute_target=cpu_cluster,
    script="train.py",
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=15,
                                     max_concurrent_runs=4)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run_exp = exp.submit(config=hyperdrive_config)
run_exp.wait_for_completion(show_output=True)

RunId: HD_5d3e03e5-7aad-4e92-90ec-7238be70d355
Web View: https://ml.azure.com/runs/HD_5d3e03e5-7aad-4e92-90ec-7238be70d355?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-267698/workspaces/quick-starts-ws-267698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-09-17T08:49:04.895624][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-09-17T08:49:05.5928670Z][SCHEDULER][INFO]Scheduling job, id='HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_0' 
[2024-09-17T08:49:05.7061306Z][SCHEDULER][INFO]Scheduling job, id='HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_1' 
[2024-09-17T08:49:05.8272635Z][SCHEDULER][INFO]Scheduling job, id='HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_2' 
[2024-09-17T08:49:05.9572804Z][SCHEDULER][INFO]Scheduling job, id='HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_3' 
[2024-09-17T08:49:06.0496689Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_1

{'runId': 'HD_5d3e03e5-7aad-4e92-90ec-7238be70d355',
 'target': 'udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2024-09-17T08:49:03.974132Z',
 'endTimeUtc': '2024-09-17T09:02:12.284323Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '697ab118-e76f-4778-9260-7f938aa19ad0',
  'user_agent': 'python/3.9.19 (Linux-5.15.0-1064-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.56.0',
  'space_size': '60',
  'best_child_run_id': 'HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_7',
  'score': '0.9171471927162368',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCl

In [9]:
import joblib
# Get your best run and save the model from that run.

best_run = run_exp.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

best_run

Best Run Id:  HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_7
Accuracy: 0.9171471927162367


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_7,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
best_run_metrics

{'Regularization Strength:': 1.0,
 'Accuracy': 0.9171471927162367,
 'Max iterations:': 90}

In [14]:
# get name of files of best_run
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-09-17-08',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [17]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_5d3e03e5-7aad-4e92-90ec-7238be70d355_7,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
#AutoML Run

from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [19]:
# create a dataframe with ds data

ds_df = ds.to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [20]:
ds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32950 non-null  int64  
 1   job             32950 non-null  object 
 2   marital         32950 non-null  object 
 3   education       32950 non-null  object 
 4   default         32950 non-null  object 
 5   housing         32950 non-null  object 
 6   loan            32950 non-null  object 
 7   contact         32950 non-null  object 
 8   month           32950 non-null  object 
 9   day_of_week     32950 non-null  object 
 10  duration        32950 non-null  int64  
 11  campaign        32950 non-null  int64  
 12  pdays           32950 non-null  int64  
 13  previous        32950 non-null  int64  
 14  poutcome        32950 non-null  object 
 15  emp.var.rate    32950 non-null  float64
 16  cons.price.idx  32950 non-null  float64
 17  cons.conf.idx   32950 non-null 

In [21]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
data = x.join(y)
train, test = train_test_split(data,train_size=0.3,random_state=1)
train.to_csv('training/train_clean_data.csv')
test.to_csv('training/test_clean_data.csv')

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [22]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir='training/',target_path='data/')
train_clean_data = TabularDatasetFactory.from_delimited_files(path=[(datastore,('data/train_clean_data.csv'))])
test_clean_data = TabularDatasetFactory.from_delimited_files(path=[(datastore,('data/test_clean_data.csv'))])


"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading training/test_clean_data.csv
Uploaded training/test_clean_data.csv, 1 files out of an estimated total of 2
Uploading training/train_clean_data.csv
Uploaded training/train_clean_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [23]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    compute_target=cpu_cluster,
    max_concurrent_iterations=4,
    training_data=train_clean_data,
    test_data=test_clean_data,
    label_column_name='y',
    n_cross_validations=2
)

In [24]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_94820f21-4bcc-442d-bd9c-95130ddedb92,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_94820f21-4bcc-442d-bd9c-95130ddedb92',
 'target': 'udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2024-09-17T09:18:13.753688Z',
 'endTimeUtc': '2024-09-17T09:46:59.498948Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'udacity-project',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-267698","workspace_name":"quick-starts-ws-267698","region":"southcentralus","compute_target":"udacit

In [25]:
# Retrieve model
best_run, fitted_model = automl_run.get_output()

# get name of files of best_run
best_run.get_file_names()

2024-09-17 09:49:22.860887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-09-17 09:49:22.860934: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-09-17 09:49:33.912584: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-09-17 09:49:40.024302: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-17 09:49:40.024511: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist


['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/274cc640/classes.interpret.json',
 'explanation/274cc640/eval_data_viz.interpret.json',
 'explanation/274cc640/expected_values.interpret.json',
 'explanation/274cc640/features.interpret.json',
 'explanation/274cc640/global_names/0.interpret.json',
 'explanation/274cc640/global_rank/0.interpret.json',
 'explanation/274cc640/global_values/0.interpret.json',
 'explanation/274cc640/local_importance_values.interpret.json',
 'explanation/274cc640/per_class_names/0.interpret.json',
 'explanation/274cc640/per_class_rank/0.interpret.json',
 'explanation/274cc640/per_class_values/0.interpret.json',
 'explanation/274cc640/rich_metadata.interpret.json',
 'explanation/274cc640/true_ys_viz.interpret.json',
 'explanation/274cc640/visualization_dict.interpret.json',
 'explanation/274cc640/ys_pred_proba_viz.interpret.json',
 'explanation/274cc640/ys_pred_viz.interpret.json',
 'explanation/7195287b/classes.interpret.json',
 'expl

In [26]:
# save best model
best_run.register_model(model_name = "model.pkl", model_path = './outputs/')
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('29', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('4', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.6933333333333332, learning_rate=0.09473736842105263, max_bin=110, max_depth=8, min_child_weight=6, min_data_in_leaf=0.003457931034482759, min_split_gain=1, n_estimators=25, n_jobs=1, num_leaves=227, problem_info=ProblemInfo(gpu_training_param_d

# Cluster clean up
compute_target.delete(